In [4]:
import sys, os
repo_start = f'../../'
sys.path.append(repo_start)

from modules.utils.imports import *
from modules.symbolic_net.model_wrapper import model_wrapper
from modules.symbolic_net.build_symbolic_net import symbolic_net
from modules.utils.training_test_split import training_test_split
from modules.analysis.generate_loss_curves import generate_loss_curves
from modules.loaders.format_data import format_data_general
from modules.generate_data.simulate_system import reaction
from modules.symbolic_net.write_terms import write_terms
from modules.symbolic_net.visualize_surface import visualize_surface

In [ ]:
dir_name = ''

In [ ]:
# Load model for analaysis
model.load(f"{dir_name}/binn_best_val_model", device=device)

# Write terms
fn = f'{dir_name}/equation.txt'
file = open(fn, 'w')
terms = write_terms(model.model.individual)

for term in terms:
    file.write(f'{term}\n')

file.close()